# ASHE Table 7

-----

### Requirements

"Annual Summary of Hours and Earnings"


#### Observations & Dimensions

The `observations` are the numbers in the percentile columns.

The required dimensions are:

* **Geography** - in the `Code` column, one letter followed by 8 digits
* **Percentiles** - 10,20,30, etc
* **Time** - year, 4 digits
* **Gender** - Male, Female, All
* **Working Pattern** - Full time, Part time, All
* **Statistics** - The "topic" of the dataset, i.e "monthly pay net etc", in the filename

-----
    
Notes:

The "statistics" seems pointless because we're looking at one file. In production there are 24 per year per ASHE table.

It's always worth getting the file out of /sources and having a look over.

In [30]:
from databaker.framework import *
import pandas as pd

tabs = loadxlstabs("./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls")

Loading ./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls which has size 846336 bytes
Table names: ['Notes', 'All', 'Male', 'Female', 'Full-Time', 'Part-Time', 'Male Full-Time', 'Male Part-Time', 'Female Full-Time', 'Female Part-Time']


In [36]:

# create a list to put each extracted tab into
tidied_sheets = []

# Iterate through all the tabs
for tab in tabs:
    
    # but skip the notes tab
    if tab.name.lower() == "notes":
        continue
    
    # Get the observations
    observations = tab.excel_ref("H5:Q5").fill(DOWN).is_not_blank()
    
    # Get the geogrpahy codes
    geography = tab.excel_ref("B5").expand(DOWN).is_not_blank()
    
    # Get gender from the tab name if it's specified, otherwise set to all
    if "male" in tab.name.lower() or "female" in tab.name.lower():
        gender = tab.name.split(" ")[0]
    else:
        gender = "all"
        
    # Get the percentiles
    percentiles = tab.filter("Percentiles").assert_one().shift(DOWN).expand(RIGHT).is_not_blank()
        
    # Get working pattern from the tab name if its specified, otherwise set to all
    if "time" in tab.name.lower():
        if " " in tab.name.lower():
            working_pattern = tab.name.lower().split(" ")[1]
        else:
            working_pattern = tab.name.lower()
    else:
        working_pattern = "all"
        
    mean = tab.filter("Mean").assert_one().fill(DOWN).is_not_blank()
    median = tab.filter("Median").assert_one().fill(DOWN).is_not_blank()
        
    # Putting it all together
    dimensions = [
        HDimConst("Gender", gender),
        HDimConst("Working Pattern", working_pattern),
        HDim(geography, "Area", DIRECTLY, LEFT),
        HDim(percentiles, "Percentile", DIRECTLY, ABOVE),
        HDim(mean, "Mean", DIRECTLY, LEFT),
        HDim(median, "Median", DIRECTLY, LEFT)
    ]
        
    tidy_sheet = ConversionSegment(tab, dimensions, observations) # < --- processing
    
    # Uncomment this, to see the selection previews
    #savepreviewhtml(tidy_sheet)
    
    # Add to the list - and convert it to a lpandas dataframe while we're at it
    tidied_sheets.append(tidy_sheet.topandas()) 
    print("Processed {}".format(tab.name))
    
# Concatenate that list of tidied sheets together with pandas
final_sheet = pd.concat(tidied_sheets)

# Display without those horrid nan values
final_sheet.fillna("")


Processed All

Processed Male

Processed Female

Processed Full-Time

Processed Part-Time

Processed Male Full-Time

Processed Male Part-Time

Processed Female Full-Time

Processed Female Part-Time


,OBS,DATAMARKER,Gender,Working Pattern,Area,Percentile,Mean,Median
0,145.2,,all,all,K02000001,10.0,555.0,460.0
1,241.1,,all,all,K02000001,20.0,555.0,460.0
2,290.2,,all,all,K02000001,25.0,555.0,460.0
3,325.3,,all,all,K02000001,30.0,555.0,460.0
4,389.5,,all,all,K02000001,40.0,555.0,460.0
...,...,...,...,...,...,...,...,...
4295,220.5,,Female,part-time,N92000002,60.0,217.7,191.7
4296,249.9,,Female,part-time,N92000002,70.0,217.7,191.7
4297,270,,Female,part-time,N92000002,75.0,217.7,191.7
4298,294.7,,Female,part-time,N92000002,80.0,217.7,191.7
